## Demo of processing user attributes in Signals including tracking Snowplow events

This notebook creates a new view using the SDK and also tracks Snowplow events to test that the attributes are processed in stream.

### Flow of data

```mermaid
flowchart LR
    sp(Snowplow Pipeline)
    stream[/Stream processing/]
    signals(Signals)

    sp --> stream
    stream --> signals
```

---

In [ ]:
from snowplow_signals import Signals
import os

try:
    from google.colab import userdata
    sp_signals = Signals(
            api_url=userdata.get('SP_API_URL'),
            api_key=userdata.get('SP_API_KEY'),
            api_key_id=userdata.get('SP_API_KEY_ID'),
            org_id=userdata.get('SP_ORG_ID'),
        )
except ImportError:
    from dotenv import load_dotenv
    load_dotenv()
    sp_signals = Signals(
        api_url=os.environ['SP_API_URL'],
        api_key=os.environ['SP_API_KEY'],
        api_key_id=os.environ['SP_API_KEY_ID'],
        org_id=os.environ['SP_ORG_ID'],
)

### Define a new attribute

This block creates a single attribute definition including the logic how it should be calculated (it's filters and aggregation).

The attribute calculates the number of add to cart ecommerce events.

In [ ]:
from snowplow_signals import (
    Attribute,
    Event,
)

count_page_views = Attribute(
    name="count_page_views",
    type="int32",
    events=[
        Event(name="page_view")
    ],
    aggregation="counter",
)

### Wrapping the attribute in a view

All attributes need to be included in views that can be considered as "tables" of attributes.

Views are immutable and versioned.

In [ ]:
from snowplow_signals import View, user_entity

view = View(
    name="page_view_attributes",
    version=1,
    entity=user_entity,
    attributes=[
        count_page_views,
    ],
)

### Testing the view

Execute the view on the last one hour of data from the atomic events table to verify that it works correctly.

In [ ]:
data = sp_signals.test(
    view=view,
    app_ids=["website"],
)
data

### Applying the view to Signals

The following block pushes the view definition to the Signals API and makes it available for processing.

In [ ]:
applied = sp_signals.apply([view])
print(f"{len(applied)} objects applied")

### Track Snowplow events to test the attributes

Track Snowplow events to test that the view is producing data.

#### 1. Create Snowplow tracker

In [ ]:
from snowplow_tracker import Snowplow

tracker = Snowplow.create_tracker(
    namespace="ns1",
    endpoint="http://localhost:8080", # TODO: choose your collector endpoint
    app_id="ai_demo", # TODO: choose app id that your Signals streaming engine is listening to
)

#### 2. Track events

In [ ]:
from snowplow_tracker import Subject, PageView

duid = "1e4a8d2a-4b3b-4b6d-8b1e-6f4f4f4f4f4f"
sid = "5f3f7a2b-0e4b-4b6d-8b1e-6f4f4f4f4f4f"

subject = Subject()
subject.set_domain_user_id(duid)
subject.set_domain_session_id(sid)

event = PageView(page_url="https://www.example.com", page_title="Example page", event_subject=subject)

tracker.track(event)
tracker.flush()

### Retrieving data

One can fetch the latest attribute values for a particular session from the view as follow.

In [ ]:
response = sp_signals.get_online_attributes(
    view,
    duid,
)

response.to_dataframe()